<a href="https://colab.research.google.com/github/JemyHo/NLPCW1/blob/Lihang/cw1/F21NL_CW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -O wikitext-filtered-full.zip "https://www.dropbox.com/scl/fi/ibd4cmixckghx6hhb361c/wikitext-filtered-full.zip?rlkey=q71cebf0k5fvvwhmcntoswzhq&dl=1"
!wget -O wikitext-filtered-10k.zip "https://www.dropbox.com/scl/fi/ek174r3sg7qjx0aa9atop/wikitext-filtered-10k.zip?rlkey=zy6jqxv6qsc16lr9qm3ki9uhf&dl=1"

--2025-10-19 22:54:16--  https://www.dropbox.com/scl/fi/ibd4cmixckghx6hhb361c/wikitext-filtered-full.zip?rlkey=q71cebf0k5fvvwhmcntoswzhq&dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.2.18, 2620:100:6017:18::a27d:212
Connecting to www.dropbox.com (www.dropbox.com)|162.125.2.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://uc3f1daa37374e2ddeae67020fe0.dl-eu.dropboxusercontent.com/cd/0/inline/CzgJY6jTCxnQe4cwSqp_RPwH0o2vNT6wO-PnaxbX4heudybQrk0KjLz2KG_XVtUat0hprzPy07Zra1DuC7A24JoC5qbOBlavEqnCAS_7yOi9aVJKV1Cjyi-JBO515u-m0s8bwCOLHgsHsHDrPIQpmBkm/file?dl=1# [following]
--2025-10-19 22:54:17--  https://uc3f1daa37374e2ddeae67020fe0.dl-eu.dropboxusercontent.com/cd/0/inline/CzgJY6jTCxnQe4cwSqp_RPwH0o2vNT6wO-PnaxbX4heudybQrk0KjLz2KG_XVtUat0hprzPy07Zra1DuC7A24JoC5qbOBlavEqnCAS_7yOi9aVJKV1Cjyi-JBO515u-m0s8bwCOLHgsHsHDrPIQpmBkm/file?dl=1
Resolving uc3f1daa37374e2ddeae67020fe0.dl-eu.dropboxusercontent.com (uc3f1daa37374e2ddeae67020fe0.dl-eu.dropbo

In [ ]:
!unzip wikitext-filtered-full.zip
!unzip wikitext-filtered-10k.zip

Archive:  wikitext-filtered-full.zip
replace wikitext-filtered-full/dataset_info.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace wikitext-filtered-full/state.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace wikitext-filtered-full/data-00000-of-00001.arrow? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
Archive:  wikitext-filtered-10k.zip
replace wikitext-filtered-10k/dataset_info.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace wikitext-filtered-10k/state.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace wikitext-filtered-10k/data-00000-of-00001.arrow? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
!pip install datasets
import datasets

In [ ]:
!pip install gensim

In [ ]:
from datasets import load_dataset, Dataset
# Store and load dataset
def load_dataset():
  wikitext_small = "wikitext-filtered-10k"
  wikitext_large = "wikitext-filtered-full"

  dataset_small = Dataset.load_from_disk(wikitext_small)
  dataset_large = Dataset.load_from_disk(wikitext_large)
  print("wikitext_small: {} docs, wikitext_large: {} docs".format(len(dataset_small), len(dataset_large)))
  return dataset_small, dataset_large

wikitext_small, wikitext_large = load_dataset()

wikitext_small: 10000 docs, wikitext_large: 859955 docs


In [ ]:
import nltk, string, os
from gensim.models import Word2Vec
nltk.download('stopwords') # Frequent buit semantically unimportant words
from nltk.corpus import stopwords
STOP = set(stopwords.words('english'))
print(f"{len(STOP)} English stopwords loaded.")

198 English stopwords loaded.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
#Step 3

def tokenize_by_space(text):
    # Minimal preprocessing
    # lowercase
    # split by spaces
    # strip punctuation
    # remove stopwords and empty tokens
    tokens = []
    for tok in text.lower().split(" "):
        tok = tok.strip(string.punctuation)
        if not tok or tok in STOP:
            continue
        tokens.append(tok)
    return tokens

def prepare_corpus(raw_ds):
    # Convert a HuggingFace dataset into a list of token lists for Word2Vec training.
    first = raw_ds[0]
    # find the right text field
    if isinstance(first, dict): # If each dataset entry is a dictionary
        key = [k for k in first.keys() if isinstance(first[k], str)][0] #find which key contains the actual text string
        docs = [ex[key] for ex in raw_ds] # Extract all text values from that key across the dataset
    else:
        docs = list(raw_ds)
        # Remove empty strings and tokenize each document using helper function
    return [tokenize_by_space(doc) for doc in docs if doc.strip()]

In [11]:
# Run the function
tokens_small = prepare_corpus(wikitext_small)
tokens_large = prepare_corpus(wikitext_large)

print("Docs (small):", len(tokens_small))
print("Docs (large):", len(tokens_large))
print("Example tokens:", tokens_small[0][:20])


Docs (small): 10000
Docs (large): 859955
Example tokens: ['senjō', 'valkyria', '3', 'unrecorded', 'chronicles', 'japanese', '戦場のヴァルキュリア3', 'lit', 'valkyria', 'battlefield', '3', 'commonly', 'referred', 'valkyria', 'chronicles', 'iii', 'outside', 'japan', 'tactical', 'role']


In [12]:
SEED = 42
WORKERS = os.cpu_count()
# Train models
def train_word2vec(sentences, name, vector_size=50, window=5, min_count=5, epochs=5, sg=0):
    print(f"▶ Training {name} | vec={vector_size}, win={window}, min={min_count}, ep={epochs}, sg={sg}")
    model = Word2Vec(
        sentences=sentences,
        vector_size=vector_size,
        window=window,
        min_count=min_count,
        sg=sg,      # 0 = CBOW, 1 = Skip-gram
        workers=WORKERS,
        seed=SEED
    )
    model.train(sentences, total_examples=len(sentences), epochs=epochs)
    print("Vocab size:", len(model.wv))
    model.save(f"w2v_{name}.model")
    print(f" Saved model: w2v_{name}.model\n")
    return model

w2v_small = train_word2vec(tokens_small, "small")
w2v_large = train_word2vec(tokens_large, "large")


▶ Training small | vec=50, win=5, min=5, ep=5, sg=0


Vocab size: 14554
 Saved model: w2v_small.model

▶ Training large | vec=50, win=5, min=5, ep=5, sg=0


Vocab size: 170080
 Saved model: w2v_large.model



In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec

# Load both trained Word2Vec models

w2v_small = Word2Vec.load("w2v_small.model")
w2v_large = Word2Vec.load("w2v_large.model")
print("Models loaded")
print("small vocab:", len(w2v_small.wv), )
print(" large vocab:", len(w2v_large.wv), )

Models loaded
small vocab: 14554
 large vocab: 170080


In [15]:
# Load WordSim-353 CSV from Google Drive

file_path = '/content/drive/My Drive/wordsim353/combined.csv'

try:
    wordsim_df = pd.read_csv(file_path)
    print("WordSim-353 data loaded successfully.")
    display(wordsim_df.head())
except FileNotFoundError:
    print(f"File not found at {file_path}")
except Exception as e:
    print(f"Error loading file: {e}")

print("Total pairs:", len(wordsim_df))


WordSim-353 data loaded successfully.


,Word 1,Word 2,Human (mean)
0,love,sex,6.77
1,tiger,cat,7.35
2,tiger,tiger,10.00
3,book,paper,7.46
4,computer,keyboard,7.62


Total pairs: 353


In [16]:
# compute cosine safely
def compute_cosine(model, w1, w2):
    w1, w2 = w1.lower(), w2.lower()
    try:
        return model.wv.similarity(w1, w2)
    except KeyError:
        return np.nan   # skip missing words

# Compute cosine for the 4 required pairs
pairs = [
    ("plane", "car"),
    ("planet", "sun"),
    ("cup", "article"),
    ("sugar", "approach")
]

# Print the results with 4 floating point
print("\n---- wikitext_small ----")
for a, b in pairs:
    print(f"{a:8s}/{b:8s} → {compute_cosine(w2v_small, a, b):.4f}")

print("\n---- wikitext_large ----")
for a, b in pairs:
   print(f"{a:8s}/{b:8s} → {compute_cosine(w2v_large, a, b):.4f}")


# Prepare for full 353-pair evaluation
def evaluate_all(model, df):
    cosines = []
    for _, row in df.iterrows():
        w1, w2 = row['Word 1'], row['Word 2']
        cos = compute_cosine(model, w1, w2)
        cosines.append(cos)
    df[f'{model}'] = cosines
    return cosines

# compute for both models
wordsim_df['cosine_small'] = [
    compute_cosine(w2v_small, w1, w2) for w1, w2 in zip(wordsim_df['Word 1'], wordsim_df['Word 2'])
]
wordsim_df['cosine_large'] = [
   compute_cosine(w2v_large, w1, w2) for w1, w2 in zip(wordsim_df['Word 1'], wordsim_df['Word 2'])
]

display(wordsim_df.head())






---- wikitext_small ----
plane   /car      → 0.6364
planet  /sun      → 0.8595
cup     /article  → 0.3383
sugar   /approach → 0.8556

---- wikitext_large ----
plane   /car      → 0.6354
planet  /sun      → 0.5992
cup     /article  → 0.1623
sugar   /approach → -0.2169


,Word 1,Word 2,Human (mean),cosine_small,cosine_large
0,love,sex,6.77,0.781484,0.579268
1,tiger,cat,7.35,0.907150,0.621646
2,tiger,tiger,10.00,1.000000,1.000000
3,book,paper,7.46,0.637400,0.597527
4,computer,keyboard,7.62,0.836236,0.403679


In [17]:
# Step 4
from scipy.stats import spearmanr

# Compute Spearman rank-order correlation
rho_small, p_small = spearmanr(wordsim_df['Human (mean)'],
                               wordsim_df['cosine_small'],
                               nan_policy='omit')

rho_large, p_large = spearmanr(wordsim_df['Human (mean)'],
                               wordsim_df['cosine_large'],
                               nan_policy='omit')

print(f"Spearman correlation (wikitext_small): {rho_small:.4f} (p={p_small:.4e})")
print(f"Spearman correlation (wikitext_large): {rho_large:.4f} (p={p_large:.4e})")


Spearman correlation (wikitext_small): 0.1549 (p=1.2205e-02)
Spearman correlation (wikitext_large): 0.6337 (p=4.8598e-41)


In [23]:
#Step 5 (if you are the first time to download the api, just run this cell, remember to clear all the ## before run)
import gensim.downloader as api

# Save it permanently to Drive
model.save("/content/drive/MyDrive/word2vec-google-news-300.model")
print("Model saved to Google Drive!")

# Load model
model = api.load("word2vec-google-news-300")
print("Model loaded!")



#"""Once you done download the api model, it should be save to google drive"""
#"""Then , put the ## back to the code"""
#"""And so when you run again, it just only call the api from google drive, no need download again"""



NameError: name 'model' is not defined

In [18]:
# Step 5
import numpy as np
from gensim.models import KeyedVectors

google_news_model = KeyedVectors.load("/content/drive/MyDrive/word2vec-google-news-300.model", mmap='r')
print("Loaded Google News model from Drive (no download needed!)")

# Safe cosine function for this KeyedVectors model
def compute_cosine_pretrained(model, w1, w2):
    w1, w2 = w1.lower(), w2.lower()
    try:
        return model.similarity(w1, w2)
    except KeyError:
        return np.nan

# Compute cosine for all pairs
wordsim_df['cosine_google'] = [
    compute_cosine_pretrained(google_news_model, w1, w2)
    for w1, w2 in zip(wordsim_df['Word 1'], wordsim_df['Word 2'])
]

# Calculate Spearman correlation
rho_google, p_google = spearmanr(wordsim_df['Human (mean)'],
                                 wordsim_df['cosine_google'],
                                 nan_policy='omit')

print(f"Spearman correlation (Google News Word2Vec): {rho_google:.4f} (p={p_google:.4e})")


Loaded Google News model from Drive (no download needed!)
Spearman correlation (Google News Word2Vec): 0.6941 (p=1.2875e-51)


In [19]:
# STEP 6

def train_and_evaluate(tokens, name, **params):
    # Train model with given parameters and compute Spearman correlation.
    model = Word2Vec(sentences=tokens, workers=os.cpu_count(), seed=42, **params)
    model.train(tokens, total_examples=len(tokens), epochs=params.get("epochs", 5))

    # Compute cosine for all 353 pairs
    df = wordsim_df.copy()
    df['cosine'] = [
        compute_cosine(model, w1, w2) for w1, w2 in zip(df['Word 1'], df['Word 2'])
    ]
    rho, _ = spearmanr(df['Human (mean)'],
                       df['cosine'],
                       nan_policy='omit')
    print(f"{name:15s} → Spearman ρ = {rho:.4f}")
    return rho

results = {}

# Example experiments
results['baseline_small'] = train_and_evaluate(tokens_small, "baseline_small",vector_size=50, window=5, min_count=5, epochs=5, sg=0)

results['larger_small'] = train_and_evaluate(tokens_small, "larger_small",vector_size=100, window=5, min_count=5, epochs=10, sg=0)

results['wide_small'] = train_and_evaluate(tokens_small, "wide_small",vector_size=50, window=10, min_count=5, epochs=10, sg=0)

results['baseline_large'] = train_and_evaluate(tokens_large, "baseline_large",vector_size=50, window=5, min_count=5, epochs=5, sg=0)

results['larger_large'] = train_and_evaluate(tokens_large, "larger_large",vector_size=100, window=5, min_count=5, epochs=10, sg=0)

results['wide_large'] = train_and_evaluate(tokens_large, "wide_large",vector_size=50, window=10, min_count=5, epochs=10, sg=0)

pd.DataFrame(list(results.items()), columns=["Model", "Spearman ρ"])


baseline_small  → Spearman ρ = 0.1473


larger_small    → Spearman ρ = 0.3151


wide_small      → Spearman ρ = 0.3502


baseline_large  → Spearman ρ = 0.6434


larger_large    → Spearman ρ = 0.6607


wide_large      → Spearman ρ = 0.6657


,Model,Spearman ρ
0,baseline_small,0.147285
1,larger_small,0.315144
2,wide_small,0.350216
3,baseline_large,0.643434
4,larger_large,0.660696
5,wide_large,0.665706


In [20]:
#step 7

from gensim.models import KeyedVectors

# load from Drive
google_news = KeyedVectors.load("/content/drive/MyDrive/word2vec-google-news-300.model", mmap='r')

def normalize_variants(w):
    # Return candidate surface forms for GoogleNews vocab
    cands = {w, w.lower(), w.capitalize(), w.title(), w.replace(" ", "_"),
             w.lower().replace(" ", "_")}
    return list(cands)

def in_vocab(model, w):
    return w in model.key_to_index

def pick_vocab_form(model, w):
    for v in normalize_variants(w):
        if in_vocab(model, v):
            return v
    return None  # not found

def top_analogy(model, a, b, c, topn=5, exclude=None):

    # Solve: a is to b as c is to ?
    # vector = b - a + c, return top 5  answer
    # Tries smart variants for vocab. Excludes source words by default.

    # Tries different variants
    a_ = pick_vocab_form(model, a)
    b_ = pick_vocab_form(model, b)
    c_ = pick_vocab_form(model, c)
    if not all([a_, b_, c_]):
        return {"query": (a,b,c), "resolved": (a_,b_,c_), "answers": [], "note": "OOV encountered"}

    # Define positive and negative words for vector arithmetic.
    neg = [a_]
    pos = [b_, c_]
    #  Compute most similar words to the resulting vector.
    res = model.most_similar(positive=pos, negative=neg, topn=topn+10)  # extra buffer
    # filter out trivial echoes of inputs
    bad = set([x for x in [a_, b_, c_] if x]) | set(normalize_variants(a) + normalize_variants(b) + normalize_variants(c))
    answers = [(w, float(s)) for (w,s) in res if w not in bad][:topn] # Each resut is a tuple (word, consince similarity)
    return {"query": (a,b,c), "resolved": (a_,b_,c_), "answers": answers, "note": ""}


In [21]:
queries_g = [
    ("man", "woman", "king"),
    ("Athens", "Greece", "Rome"),
    ("reading", "read", "playing"),
    ("Greece", "souvlaki", "Italy"),
    ("airplane", "propeller", "car"),
]

# Print results
for a,b,c in queries_g:
    out = top_analogy(google_news, a,b,c, topn=5)
    print(f"{a} : {b} :: {c} : ?")
    print(" resolved tokens:", out["resolved"])
    if out["answers"]:
        for i,(w,s) in enumerate(out["answers"],1):
            print(f"  {i}. {w:30s}  ({s:.4f})")
    else:
        print("  [OOV or no results]", out["note"])
    print()


man : woman :: king : ?
 resolved tokens: ('Man', 'Woman', 'king')
  1. queen                           (0.6718)
  2. monarch                         (0.5842)
  3. princess                        (0.5416)
  4. prince                          (0.5270)
  5. kings                           (0.5232)

Athens : Greece :: Rome : ?
 resolved tokens: ('Athens', 'greece', 'rome')
  1. pablo                           (0.6213)
  2. samuel                          (0.6207)
  3. omar                            (0.6192)
  4. cro                             (0.6153)
  5. zim                             (0.6134)

reading : read :: playing : ?
 resolved tokens: ('Reading', 'read', 'Playing')
  1. played                          (0.4643)
  2. play                            (0.4147)
  3. Played                          (0.3971)
  4. reads                           (0.3753)
  5. boldly_proclaims                (0.3727)

Greece : souvlaki :: Italy : ?
 resolved tokens: ('greece', 'Souvlaki', 'Italy')
  1. 

In [22]:
queries_h = [
    ("man", "woman", "computer programmer"),
    ("man", "woman", "superstar"),
    ("man", "woman", "guitarist"),
    ("man", "woman", "boss"),

]

for a,b,c in queries_h:
    out = top_analogy(google_news, a,b,c, topn=5)
    print(f"{a} : {b} :: {c} : ?")
    print(" resolved tokens:", out["resolved"])
    if out["answers"]:
        for i,(w,s) in enumerate(out["answers"],1):
            print(f"  {i}. {w:30s} ({s:.4f})")
    else:
        print("  [OOV or no results]", out["note"])
    print()


man : woman :: computer programmer : ?
 resolved tokens: ('Man', 'Woman', 'computer_programmer')
  1. homemaker                      (0.5471)
  2. electrical_engineer            (0.5178)
  3. mechanical_engineer            (0.5084)
  4. registered_nurse               (0.4998)
  5. schoolteacher                  (0.4905)

man : woman :: superstar : ?
 resolved tokens: ('Man', 'Woman', 'superstar')
  1. megastar                       (0.6378)
  2. diva                           (0.5861)
  3. star                           (0.5716)
  4. songstress                     (0.5471)
  5. singing_sensation              (0.5318)

man : woman :: guitarist : ?
 resolved tokens: ('Man', 'Woman', 'Guitarist')
  1. Vocalist                       (0.6885)
  2. Bassist                        (0.6669)
  3. Drummer                        (0.6266)
  4. Musician                       (0.6126)
  5. Keyboardist                    (0.6004)

man : woman :: boss : ?
 resolved tokens: ('Man', 'Woman', 'boss')
  1.